# Loops

**PRELIMINARY SITE DRAFT. MUCH WORK TO BE DONE!**


For loops can be useful when you want to iterate a process in R --- e.g., run a simulation. This tutorial explains how to write for loops and shows how to use them to run Monte Carlo simulations. For loops are neat, but it's worth emphasizing that you should avoid them and instead use [vectorization](http://www.noamross.net/blog/2014/4/16/vectorization-in-r--why.html) --- which is much faster --- when possible. 

## Writing a for loop

Let's start with a very simple example. Let's say you 
have the following vector `v1` with individuals' height in centimeters:

In [ ]:
v1 <- c(175, 182, 150, 187, 165)

We can convert the values in `v1` from centimeters to meters using a for loop: 

In [ ]:
v2 <- rep(NA, 5)               #create vector v2 with NA values 
for(i in 1:5) {                #loop over elements in v1 and store in v2  
    v2[i] <- v1[i] / 100
}
v2                             #v2 after the for loop

Note that we could have done this using
[vectorization](http://www.noamross.net/blog/2014/4/16/vectorization-in-r--why.html),
which is more compact and faster (when we have a lot of data): 

In [ ]:
v2 <- v1 / 100

However, the example illustrates the following points about writing a
for loop:

1. Begin by creating an object that can store the results of your for
loop.  In the example above, we created `v2` for this purpose. With
vectors, we need to specify how many elements we want to eventually
store, in this case 5. (This is not true if you wanted to store the
results in a list.)  

2. The basic structure of the loop usually is: 

In [ ]:
n <- 5
for(i in 1:n) {
    #commands to execute for iteration i
}

Here `n` represents the number of times you want to iterate the loop. The loop 
will run from 1 to `n` by an integer count. If you instead wanted the loop to iterate
from 1 to `n` but skip every other number you could use `seq(1, n, by = 2)` in place of 
`1:n`. 

3\.&nbsp;Within the for loop we want to save the output of each iteration to an element of 
the vector (or list) we created initially (in this case `v2`).
\
\

Here's a more general approach accomplishing the same thing, but where 
we keep the number of iterations flexible depending on how many elements 
`v1` contains: 

In [ ]:
v1 <- c(175, 182, 150, 187, 165)
n <- length(v1)
v2 <- rep(NA, n)
for(i in 1:n) {
    v2[i] <- v1[i] / 100
}
v2

Of course, you can store outputs from the for loop in a vector within a data 
frame. Say we had the following data frame with names and heights: 

In [ ]:
ppl <- data.frame(person = letters[1:5], height_cm = v1)
ppl

Let's add a variable that expresses height in inches instead:

In [ ]:
ppl$height_inch <- NA                                     #add variable of NAs
n <- nrow(ppl)                                            #get number of observations to loop over
for(i in 1:n){
    ppl$height_inch[i] <- ppl$height_cm[i] * 0.393701
}
ppl

Note that when adding a constant or `NA` values to a vector within a
data frame, `R` (correctly) assumes that you want to add this constant to every 
element of the variable, so you don't need to specify how many times you want to 
add `NA` in `ppl$height_inch <- NA`. 

## An appliction

For loops can be used to carry out Monte Carlo simulations. In the
example below, we'll draw repeated samples from a population,
calculate the mean for each sample, and test whether we on average do
a good job of estimating the population mean. 

Say the population consists of 10 individuals with the following heights: 

In [ ]:
v <- c(175, 182, 150, 187, 165, 177, 200, 198, 157, 165)
mean(v)    #population mean

Unfortunately, for whatever reason, we do not know the heights of all
of these individuals. We can only (randomly) sample 5 of them. From
this random sample of five individuals we estimate the height of all
10 individuals. We can draw a sample of 5 from `v` and take the mean
of this sample using the following code:

In [ ]:
v <- c(175, 182, 150, 187, 165, 177, 200, 198, 157, 165)
smpl <- sample(v, 5)
mean(smpl)

Would we on average expect to estimate the mean of the population
accurately?  Let's use a Monte Carlo simulation to find out. We'll
draw 10,000 random samples of five from `v` and take the mean of each
of these samples. With an unbiased estimator we would, on average,
expect the sample estimate to equal the population parameter of
interest.

In [ ]:
n <- 10000
smpl_means <- rep(NA, n)
for(i in 1:n){
    smpl <- sample(v, 5)
    smpl_means[i] <- mean(smpl)
}

mean(smpl_means)

The mean of the sample means (`r mean(smpl_means)`) is very close to
the population mean (`r mean(v)`): on average, we're accurately
estimating the population mean with our random sample of five
individuals. 

Note, though, that in some cases our estimate is quite far from the population
mean. To illustrate this, we can plot a histogram of the sample means: 

In [4]:
require(ggplot2)
ggplot(data.frame(smpl_means), aes(x = smpl_means)) +
    geom_histogram(binwidth = 2) +
    geom_vline(xintercept = mean(v), color = "red", linetype = 2) +
    xlab("Sample mean (n = 5)") +
    ylab("Number of samples") + 
    theme_bw()

Loading required package: ggplot2



ERROR: Error in data.frame(smpl_means): object 'smpl_means' not found


The dashed red line indicates the population mean. While our sample
estimates are centered around this mean (good news!), the range of the
estimates is quite large. In fact, about 10% of the time we'd get an
estimate of the mean that is either almost 9 centimeters (3.5 inches)
below the actual mean or almost 9 centimeters above the actual mean:

In [ ]:
quantile(smpl_means, probs = c(0.05, 0.95))

## Don't Loop Over Big Vectors / Data 


**CAUTION:** Do not loop over your dataset rows

Loops are powerful, but one thing you almost never want to do is loop over the rows of your dataset. The reason is that looping is **much** slower than doing an operation using vector math (a practice called "vectorization"). 

To illustrate, suppose I want to add up two vectors, each with 1,000,000 entries:

In [4]:
v1 <- rnorm(1000000)
v2 <- rnorm(1000000)

# add up with vector math:

vector_time <- system.time(v1 + v2)
vector_time

   user  system elapsed 
  0.002   0.000   0.001 

Note I have to put them in a function to do it because of how 

In [5]:
# Now add them up in a loop.

looped <- function(v1, v2){
    # Add up in a loop
    for (i in 1:1000000) {
        v1[i] = v1[i] + v2[i]
    }
    return(v1)
}

looped_time <- system.time(looped(v1, v2))
looped_time

   user  system elapsed 
  0.070   0.003   0.073 

In [7]:
# Looped took about this many time longer:

round(looped_time[["elapsed"]] / vector_time[["elapsed"]])


[1] 73

So... yeah. 73x slower. Don't do it. 

Also, if that code looks weird, it's because the result of `system.time` is being stored in a list, which is an interesting data structure we sadly don't have time to cover, but which you can read about [here](lists.ipynb))

## Exercises


1. Use a for loop to take the square root of each value in the following 
vector: `vec1 <- c(4, 9, 81, 100, 1000, 10^6)`. Save the results to a new 
vector called `vec2`. 

2. Monte Carlo Simulation: Imagine that the values in the vector `pop`
below represent vote shares for a presidential candidate across the
3,144 counties in the United States. If we were to take a sample of 50
counties and estimate mean support for the presidential candidate,
would we, on average, estimate the vote share across all counties
accurately? (Don't worry about the fact that we really should be
weighing counties by their population size to estimate overall
support.) Draw 10,000 samples of 50 counties from `pop` and estimate
mean support for each sample, saving each mean estimate into a vector
called `smpl_means`. How does the mean of the sample means compare to 
the population mean? Do we, on average, do a good job of estimating the 
population mean? 

<div class="indent">

In [ ]:
pop <- runif(n = 3144, min = 0, max = 1)

</div>





	